<a href="https://colab.research.google.com/github/SIDIBEMoussa/NLP-Trainings/blob/main/03_Exercice_LM_classification_and_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [344]:
import pandas as pd
from random import seed
from warnings import filterwarnings

filterwarnings('ignore')

seed(1999)

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [345]:
import pathlib, os

In [346]:
path = "/content/datasets/lda_sports_politics_docs"

file_directory = pathlib.Path(path).exists()

if not file_directory:

    !git clone https://github.com/selva86/datasets.git



text_files = os.listdir(path)

In [347]:
text_files

['shinzo abe.txt',
 'pizza.txt',
 'lee quan yew.txt',
 'queen elizabeth.txt',
 'cricket.txt',
 'idli.txt',
 'barack obama.txt',
 'baseball.txt',
 'badminton.txt',
 'noodles.txt',
 'tipu sultan.txt',
 'table tennis.txt',
 'pasta.txt',
 'narendra modi.txt',
 'dosa.txt']

In [348]:
articles=[]

for filename in text_files:

    article = pathlib.Path(path+'/'+filename).read_text()

    articles.append(article)

In [349]:
articles[0]

"Shinzō Abe (Abe Shinzō, IPA: born 21 September 1954) is the current Prime Minister of Japan, re-elected to the position in December 2012. Abe is also the President of the Liberal Democratic Party (LDP).\nAbe served for a year as Prime Minister, from 2006 to 2007. Hailing from a politically prominent family, at age 52, Abe became Japan's youngest post-war prime minister, and the first to be born after World War II, when he was elected by a special session of the National Diet in September 2006. Abe resigned on 12 September 2007, for health reasons. Abe was replaced by Yasuo Fukuda, beginning a string of five Prime Ministers, none of whom retained office for more than sixteen months, before Abe staged a political comeback.\nOn 26 September 2012, Abe defeated former Minister of Defense Shigeru Ishiba, in a run-off vote, to win the LDP Presidential Election. Following the LDP's landslide victory in the 2012 general election, Abe became the Prime Minister again. Abe is the first former Pri

In [350]:
from pprint import pprint # for printing objects nicely

from gensim import corpora, models
from gensim.utils import simple_preprocess

# from gensim.parsing.preprocessing import STOPWORDS 
# Instead of the gensim english stopwords, we use nltk's German stopwords
from nltk.corpus import stopwords

from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *

import numpy as np

from random import choice

np.random.seed(1234)

stemmer = SnowballStemmer('english')
german_stop_words = set(stopwords.words('english'))


def lemmatize_stemming(text):
  return stemmer.stem(text)

def preprocess(text):
  result = [lemmatize_stemming(token) 
            for token in simple_preprocess(text)  
            if token not in german_stop_words and len(token) > 3]
  return result

In [351]:
print('original document: ')
article = choice(articles)
print(article)

# This time, we don't care about punctuations as tokens (Can you think why?):
print('original document, broken into words: ')
words = [word for word in article.split(' ')]
print(words)
print("Vocabulary size of the original article:", len(set(words)))

# now let's see what happens when we pass the article into our preprocessing 
# method:
print('\n\n tokenized and lemmatized document: ')
preprocessed_article = preprocess(article)
print(preprocessed_article)
print("Vocabulary size after preprocessing:", len(set(preprocessed_article)))

original document: 
Pizza is a flatbread generally topped with tomato sauce and cheese and baked in an oven. It is commonly topped with a selection of meats, vegetables and condiments. The term was first recorded in the 10th century, in a Latin manuscript from Gaeta in Central Italy. The modern pizza was invented in Naples, Italy, and the dish and its variants have since become popular in many areas of the world.
In 2009, upon Italy's request, Neapolitan pizza was safeguarded in the European Union as a Traditional Speciality Guaranteed dish. The Associazione Verace Pizza Napoletana (the True Neapolitan Pizza Association) is a non-profit organisation founded in 1984 with headquarters in Naples. It promotes and protects the "true Neapolitan pizza".
Pizza is sold fresh or frozen, either whole or in portions, and is a common fast food item in Europe and North America. Various types of ovens are used to cook them and many varieties exist. Several similar dishes are prepared from ingredients

In [352]:
processed_docs = list(map(preprocess, articles))
processed_docs[:2]

[['shinzō',
  'shinzō',
  'born',
  'septemb',
  'current',
  'prime',
  'minist',
  'japan',
  'elect',
  'posit',
  'decemb',
  'also',
  'presid',
  'liber',
  'democrat',
  'parti',
  'serv',
  'year',
  'prime',
  'minist',
  'hail',
  'polit',
  'promin',
  'famili',
  'becam',
  'japan',
  'youngest',
  'post',
  'prime',
  'minist',
  'first',
  'born',
  'world',
  'elect',
  'special',
  'session',
  'nation',
  'diet',
  'septemb',
  'resign',
  'septemb',
  'health',
  'reason',
  'replac',
  'yasuo',
  'fukuda',
  'begin',
  'string',
  'five',
  'prime',
  'minist',
  'none',
  'retain',
  'offic',
  'sixteen',
  'month',
  'stage',
  'polit',
  'comeback',
  'septemb',
  'defeat',
  'former',
  'minist',
  'defens',
  'shigeru',
  'ishiba',
  'vote',
  'presidenti',
  'elect',
  'follow',
  'landslid',
  'victori',
  'general',
  'elect',
  'becam',
  'prime',
  'minist',
  'first',
  'former',
  'prime',
  'minist',
  'return',
  'offic',
  'sinc',
  'shigeru',
  'yoshi

In [353]:
dictionary = corpora.Dictionary(processed_docs)

In [354]:
for idx, (k, v) in enumerate(dictionary.iteritems()):
    print(k, v)
    if idx >= 10:
        break

0 also
1 becam
2 begin
3 born
4 coalit
5 comeback
6 current
7 decemb
8 defeat
9 defens
10 democrat


### Model Hyperparameter tuning

In [355]:
## Model hyper parameters:

## These are the dictionary preparation parameters:
filter_tokens_if_container_documents_are_less_than = 1
filter_tokens_if_appeared_percentage_more_than = 0.5
keep_the_first_n_tokens=100000

## and the LDA Parameters: 
num_of_topics = 4

In [356]:
dictionary.filter_extremes(
        no_below=filter_tokens_if_container_documents_are_less_than, 
        no_above=filter_tokens_if_appeared_percentage_more_than, 
        keep_n=keep_the_first_n_tokens)

In [357]:
def optimization(processed_docs,min_topic=4, max_topic=10,more_than_number=10,less_than_range=20):

    model_results = {#'Validation_Set': [],
                 'Topics': [],
                 'More_than': [],
                 'Less_than': [],
                 'Perplexity': []
                }
    keep_the_first_n_tokens = 10000
    
    dictionary = corpora.Dictionary(processed_docs)

    for more_than in np.linspace(0,1,more_than_number):

      for less_than in range(1,less_than_range):

          dictionary.filter_extremes(
          no_below = less_than, 
          no_above = more_than, 
          keep_n = keep_the_first_n_tokens)
          bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

          
          if list(dictionary.keys())!=[]:
              print(list(dictionary.keys()))
              lda_model = models.LdaMulticore(bow_corpus, 
                                    num_topics=4, 
                                    id2word=dictionary,
                                    passes=10, 
                                    workers=2)
              
              model_results["More_than"].append(more_than)
              model_results["Less_than"].append(less_than)
              model_results["Perplexity"].append(lda_model.log_perplexity(bow_corpus))
    
    return model_results

In [358]:
model_results=optimization(processed_docs,min_topic=2, max_topic=10,more_than_number=10,less_than_range=10)

In [359]:
model_results

{'Topics': [], 'More_than': [], 'Less_than': [], 'Perplexity': []}

In [360]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

#bow_corpus[:1]

In [361]:
# randomly choose an article from the corpus:
sample_bow_doc = choice(bow_corpus)

print('The processed bag-of-word document is just pairs of (word_id, # of occurnces) and looks like this:')
print(sample_bow_doc, '\n\n')

print ('We peek in the dictionary: for each word_id, we get its assigned word:')
for word_id, word_freq in sample_bow_doc:
  real_word = dictionary[word_id]
  print(f'Word {word_id} ("{real_word}") appears {word_freq} time.')

The processed bag-of-word document is just pairs of (word_id, # of occurnces) and looks like this:
[(14, 1), (107, 3), (125, 1), (131, 2), (149, 1), (168, 2), (181, 1), (230, 1), (311, 1), (312, 1), (313, 1), (314, 1), (315, 1), (316, 1), (317, 1), (318, 2), (319, 1), (320, 1), (321, 1), (322, 2), (323, 1), (324, 1), (325, 1), (326, 3), (327, 1), (328, 1), (329, 1), (330, 1), (331, 1), (332, 1), (333, 1), (334, 1), (335, 1), (336, 1), (337, 1), (338, 1), (339, 1), (340, 1), (341, 1), (342, 1), (343, 1), (344, 1), (345, 1), (346, 1), (347, 1), (348, 1)] 


We peek in the dictionary: for each word_id, we get its assigned word:
Word 14 ("five") appears 1 time.
Word 107 ("popular") appears 3 time.
Word 125 ("tradit") appears 1 time.
Word 131 ("variant") appears 2 time.
Word 149 ("countri") appears 1 time.
Word 168 ("known") appears 2 time.
Word 181 ("peopl") appears 1 time.
Word 230 ("four") appears 1 time.
Word 311 ("amongst") appears 1 time.
Word 312 ("anoth") appears 1 time.
Word 313 ("

## LDA model using Bag-of-words

In [362]:
lda_model = models.LdaMulticore(bow_corpus, 
                                num_topics=num_of_topics, 
                                id2word=dictionary, 
                                passes=10, 
                                workers=2)

In [363]:
for idx, topic in lda_model.print_topics(num_of_topics):
    print(f'Topic: {idx} \t Words: {topic}')

Topic: 0 	 Words: 0.022*"minist" + 0.018*"noodl" + 0.016*"prime" + 0.016*"elect" + 0.010*"septemb" + 0.008*"serv" + 0.008*"general" + 0.008*"offic" + 0.008*"sinc" + 0.008*"gujarat"
Topic: 1 	 Words: 0.023*"pizza" + 0.012*"ball" + 0.012*"tabl" + 0.009*"dish" + 0.009*"common" + 0.009*"use" + 0.009*"player" + 0.009*"itali" + 0.009*"neapolitan" + 0.006*"mani"
Topic: 2 	 Words: 0.015*"tipu" + 0.015*"dosa" + 0.012*"mysor" + 0.007*"team" + 0.007*"father" + 0.006*"presid" + 0.006*"democrat" + 0.006*"unit" + 0.006*"primari" + 0.006*"state"
Topic: 3 	 Words: 0.024*"pasta" + 0.016*"shape" + 0.010*"form" + 0.008*"known" + 0.008*"also" + 0.008*"play" + 0.008*"fresh" + 0.008*"elizabeth" + 0.006*"name" + 0.006*"dri"


## TF / IDF

In [364]:
# initialize a tfidf from our corpus
tfidf = models.TfidfModel(bow_corpus)

# apply it on our corpus 
tfidf_corpus = tfidf[bow_corpus]

pprint(tfidf_corpus[0][:10])

[(0, 0.03278411436243488),
 (1, 0.09458263214900205),
 (2, 0.09689176633988147),
 (3, 0.07861485368951274),
 (4, 0.09689176633988147),
 (5, 0.09689176633988147),
 (6, 0.0575843394951251),
 (7, 0.07209154120719125),
 (8, 0.07209154120719125),
 (9, 0.09689176633988147)]


In [365]:
# the new tfidf corpus is just our corpus - but transformed. It has the same size of documents:
assert len(bow_corpus) == len(tfidf_corpus)

Now let's apply LDA on the tfidf corpus, with the same amount of topics.

You can play with the # of passes, if the model doesn't converge properly

In [366]:
lda_model_tfidf = models.LdaMulticore(tfidf_corpus, 
                                      num_topics=num_of_topics, 
                                      id2word=dictionary, 
                                      passes=10, 
                                      workers=4)

In [367]:
for idx, topic in lda_model_tfidf.print_topics(num_of_topics):
    print(f'Topic: {idx} \t Word: {topic}')

Topic: 0 	 Word: 0.004*"noodl" + 0.003*"team" + 0.003*"racquet" + 0.003*"play" + 0.003*"elizabeth" + 0.003*"side" + 0.003*"queen" + 0.003*"saint" + 0.003*"badminton" + 0.003*"court"
Topic: 1 	 Word: 0.003*"gujarat" + 0.003*"modi" + 0.003*"base" + 0.003*"plate" + 0.003*"improv" + 0.003*"around" + 0.003*"home" + 0.003*"bat" + 0.003*"run" + 0.002*"minist"
Topic: 2 	 Word: 0.004*"dosa" + 0.004*"minist" + 0.003*"prime" + 0.003*"elect" + 0.003*"popular" + 0.003*"id" + 0.002*"septemb" + 0.002*"obama" + 0.002*"primari" + 0.002*"unit"
Topic: 3 	 Word: 0.004*"pasta" + 0.004*"pizza" + 0.004*"tipu" + 0.003*"tabl" + 0.003*"mysor" + 0.003*"shape" + 0.002*"ball" + 0.002*"neapolitan" + 0.002*"bounc" + 0.002*"fresh"


## Inference

Now that we have a topic-modeler, let's use it on one of the articles.

In [368]:
# randomly pick an article:
test_doc = choice(range(len(processed_docs)))
processed_docs[test_doc][:10]

['kuan',
 'gcmg',
 'spmj',
 'born',
 'harri',
 'kuan',
 'septemb',
 'march',
 'inform',
 'known']

Using the original BOW model:

In [369]:
for index, score in sorted(lda_model[bow_corpus[test_doc]], key=lambda tup: -1*tup[1]):
    print(f"Topic match score: {score} \nTopic: {lda_model.print_topic(index, num_of_topics)}")


Topic match score: 0.9920631647109985 
Topic: 0.015*"tipu" + 0.015*"dosa" + 0.012*"mysor" + 0.007*"team"


And with the TF/IDF model:

In [370]:
for index, score in sorted(lda_model_tfidf[bow_corpus[test_doc]], key=lambda tup: -1*tup[1]):
    print("Topic match score: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, num_of_topics)))

Topic match score: 0.9921205043792725	 
Topic: 0.004*"dosa" + 0.004*"minist" + 0.003*"prime" + 0.003*"elect"


In [371]:
print('Perplexity: ', lda_model.log_perplexity(bow_corpus)) 
print('Perplexity TFIDF: ', lda_model_tfidf.log_perplexity(bow_corpus)) 

Perplexity:  -6.707541112476166
Perplexity TFIDF:  -8.128765982801298


In [372]:
import matplotlib.pyplot as plt

def optimize_topic(processed_docs,max_num_topic):

    perplexity_bow = {}
    perplexity_tfidf = {}

    keep_the_first_n_tokens = 10000
    
    dictionary = corpora.Dictionary(processed_docs)
    bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

    # initialize a tfidf from our corpus
    tfidf = models.TfidfModel(bow_corpus)

    # apply it on our corpus 
    tfidf_corpus = tfidf[bow_corpus]

    for num_topic in range(2, max_num_topic):

          lda_model = models.LdaMulticore(bow_corpus, 
                                num_topics=num_topic, 
                                id2word=dictionary,
                                passes=10, 
                                workers=2)
          
          lda_model_tfidf = models.LdaMulticore(tfidf_corpus, 
                                      num_topics=num_topic, 
                                      id2word=dictionary, 
                                      passes=10, 
                                      workers=4)
          
          perplexity_bow[num_topic] = lda_model.log_perplexity(bow_corpus)

          perplexity_tfidf[num_topic] = lda_model_tfidf.log_perplexity(tfidf_corpus)


    plt.plot(list(perplexity_bow.keys()),list(perplexity_bow.values()),"b",label = "bow")
    plt.plot(list(perplexity_tfidf.keys()),list(perplexity_tfidf.values()), "g", label = "tfidf")

    plt.show()
    
    return 0
          

In [ ]:
optimize_topic(processed_docs,max_num_topic = 20)

### Exercise - inference

Now try it on a new document!

Go to a news website, such as [orf.at](https://orf.at/) and copy an article of your choice here:

In [ ]:
#https://www.lemonde.fr/en/opinion/article/2022/12/27/speaking-the-truth-about-immigration_6009281_23.html

unseen_document = """You can share an article by clicking on the share icons at the top right of it. 
The total or partial reproduction of an article, without the prior written authorization of Le Monde, is strictly forbidden. 
For more information, see our Terms and Conditions. 
For all authorization requests, contact droitsdauteur@lemonde.fr. 

The French reversal may have only a remote connection to migration issues. In each of the target countries,
visa restrictions fueled strong criticism of France. But in a context of rising anti-French sentiment throughout
Africa and faced with the risk of emerging countries leaving the Western bloc, Paris was able to give precedence
to these fundamental concerns over the math of deportations. Other bilateral issues (Western Sahara, gas, etc.),
as well as the difficulty of giving in to Morocco without doing the same for Algeria, may also have played roles.
The equation cannot be summed up as a simple trade-off between visas and deportations, because relations between 
France and its former colonies, marked by the presence of large communities of Maghrebi origin, are a matter of 
both foreign policy and domestic affairs."""

bow_vector = dictionary.doc2bow(preprocess(unseen_document))

print("Simply printing the lda_model output would look like this:")
pprint(lda_model[bow_vector])

print("\n\nSo let's make it nicer, by printing the topic contents:")
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))


## Visualization

In [ ]:
! pip install pyLDAvis

In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

bow_lda_data = gensimvis.prepare(lda_model, bow_corpus, dictionary)

pyLDAvis.display(bow_lda_data)